In [430]:
import pandas as pd
from lib import standardize_item_numbers
from scipy.stats import chi2_contingency
import statsmodels.api as sm
import numpy as np
import statsmodels.stats.multitest as smm

In [431]:
def clean():
    dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")
    dfa = dfa.pipe(standardize_item_numbers, ["item_number"])

    dfb = pd.read_csv("../data/real_time_crime_center/rtcc.csv")
    dfb["rtcc_requested"] = "1"
    dfb = dfb.pipe(standardize_item_numbers, ["item_number"])

    df = pd.merge(dfa, dfb, on="item_number")
    return df

In [432]:
df = clean()

C:\Users\ayyubi\AppData\Local\Temp\ipykernel_39876\153542535.py:2: DtypeWarning: Columns (13,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")


In [433]:
def clean_offender_race(df):
    df.loc[:, "offender_race"] = df.offender_race.str.lower().str.strip()
    return df

In [434]:
df = df.pipe(clean_offender_race)

In [435]:
top_10_charge_descs_sr = (pd.Series(df.groupby("charge_description")
                                                .rtcc_requested.value_counts()
                                                .sort_values(ascending=False).head(10))
)
top_10_charge_descs_df = (pd.DataFrame(top_10_charge_descs_sr).unstack()
                                                              .reset_index()
)

top_10_charge_desc_list = [x for x in top_10_charge_descs_df["charge_description"]]

In [436]:
df = df[df.charge_description.isin(top_10_charge_desc_list)]

In [437]:
df = df[~((df.charge_description.fillna("") == ""))]
df.loc[:, "offender_race"] = df.offender_race.fillna("").str.replace(r"^$", "black", regex=True)

In [438]:
df = df[["offender_race", "charge_description"]]

charges = pd.get_dummies(df["charge_description"])
df = pd.concat([df, charges], axis=1)

races = pd.get_dummies(df["offender_race"])
df = pd.concat([df, races], axis=1)

In [439]:
df = df.drop(columns=["charge_description", "amer. ind.", "hispanic", "asian", "offender_race",])

In [441]:
races = ['black', 'white', 'unknown']
results_c2 = {}
for race in races:
    results_c2[race] = {}
    for col in df:
        if pd.api.types.is_numeric_dtype(df[col]):
            CrosstabResult = pd.crosstab(index=df[race], columns=df[col])
            ChiSqResult = chi2_contingency(CrosstabResult)
            results_c2[race][col] = ChiSqResult[1]
results = pd.DataFrame.from_dict(results_c2, orient="index")
results

,AGG. BATTERY,AGG. CRIMINAL DAMAGE,AGGRAVATED ASSAULT WITH A FIREARM,ARMED ROBBERY,SIMPLE BURGLARY,SIMPLE CRIMINAL DAMAGE TO PROPERTY,SIMPLE ROBBERY,THEFT,THEFT OF A MOTOR VEHICLE,USE OF FIREARM IN ROBBERY,black,unknown,white
black,1.830304e-13,7.589706e-25,1.709513e-09,1.177811e-56,1.816258e-14,4.304823e-33,2.594596e-24,0.009234,4.946784e-10,1.276295e-39,0.000000e+00,0.000000e+00,2.855272e-132
white,4.231211e-04,4.905849e-04,3.286511e-02,3.283886e-02,2.076675e-01,4.380683e-01,2.133689e-01,0.000327,3.615904e-01,1.520485e-04,2.855272e-132,6.804999e-17,0.000000e+00
unknown,4.341797e-11,1.184539e-34,5.873861e-13,6.568339e-54,1.093990e-17,4.607720e-34,1.698802e-29,0.000001,7.117821e-12,4.350754e-34,0.000000e+00,0.000000e+00,6.804999e-17


In [442]:
count_data_black = df.groupby(['black']).sum()
count_data_black = count_data_black.reset_index()

results_dict = {}

for col in count_data_black.columns[:]:
    model = sm.GLM(count_data_black[col], count_data_black[['black']], family=sm.families.Poisson())
    result = model.fit()
    results_dict[col] = result.pvalues[0]
    

results_df = pd.DataFrame.from_dict(results_dict, orient='index', columns=['p_value'])
results_df
# perform multiple comparison correction using FDR method
results_df['p_adjusted'] = smm.multipletests(results_df['p_value'], method='fdr_bh')[1]

# filter for statistically significant results
significant_results_df = results_df[results_df['p_adjusted'] < 0.05]
significant_results_df

,p_value,p_adjusted
AGG. BATTERY,0.0,0.0
AGG. CRIMINAL DAMAGE,0.0,0.0
AGGRAVATED ASSAULT WITH A FIREARM,0.0,0.0
ARMED ROBBERY,0.0,0.0
SIMPLE BURGLARY,0.0,0.0
SIMPLE CRIMINAL DAMAGE TO PROPERTY,0.0,0.0
SIMPLE ROBBERY,0.0,0.0
THEFT,0.0,0.0
THEFT OF A MOTOR VEHICLE,0.0,0.0
USE OF FIREARM IN ROBBERY,0.0,0.0
